<a href="https://colab.research.google.com/github/smitdesai1010/COVID-19-Detection/blob/master/COVID_19_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import os
import shutil
import zipfile

!echo > /content/kaggle.json

token = {"username":"smitdesai1010","key":"41f928f2f4a0f0b462ed2ec45f383c82"}
with open('/content/kaggle.json', 'w') as file:
    json.dump(token, file)

!cp /content/kaggle.json ~/.kaggle/kaggle.json

!kaggle config set -n path -v /content
!chmod 600 /root/.kaggle/kaggle.json

!kaggle datasets download -d amanullahasraf/covid19-pneumonia-normal-chest-xray-pa-dataset

zf = zipfile.ZipFile('/content/datasets/amanullahasraf/covid19-pneumonia-normal-chest-xray-pa-dataset/covid19-pneumonia-normal-chest-xray-pa-dataset.zip') 
zf.extractall()

!rm -rf 'datasets' #Deleting the the downloaded Zip files after extracting it 


def create_dataset(name):
  org_path = '/content/'+ name
  train_path = '/content/train/'+ name
  val_path = '/content/validation/'+ name
  test_path = '/content/test/'+ name

  if not os.path.exists(train_path):
    os.mkdir(train_path)
    os.mkdir(val_path)
    os.mkdir(test_path)

  path, dirs, files = next(os.walk(org_path))
  file_count = len(files)
  i = 0  
    
  for file in os.listdir(org_path):
    
    if not os.path.isfile(os.path.join(org_path,file)):
      continue

    if i < file_count*0.75:
     shutil.copy2(os.path.join(org_path,file), os.path.join(train_path,file))
    
    elif i > file_count*0.9:
     shutil.copy2(os.path.join(org_path,file), os.path.join(test_path,file))

    else:
     shutil.copy2(os.path.join(org_path,file), os.path.join(val_path,file))
     
    i = i + 1 



if not os.path.exists('/content/train'):
  !mkdir /content/train
  !mkdir /content/validation
  !mkdir /content/test
 
create_dataset('covid')
create_dataset('normal')
create_dataset('pneumonia')

!rm -rf 'covid'
!rm -rf 'normal'
!rm -rf 'pneumonia'

https://medium.com/@mrgarg.rajat/training-on-large-datasets-that-dont-fit-in-memory-in-keras-60a974785d71

In [ ]:
import tensorflow as tf
import cv2

import matplotlib.pyplot as plt
import numpy as np

from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop

# specifically for cnn
from keras.layers import Dropout, Flatten,Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization

from keras.preprocessing.image import ImageDataGenerator

In [ ]:
img_dim = 256
batch_size = 64

train_generator = ImageDataGenerator(
    rescale = 1. / 255 ,
    horizontal_flip = True
)

val_generator = ImageDataGenerator(
    rescale = 1. / 255 ,
    horizontal_flip = True
)

test_generator = ImageDataGenerator(
    rescale = 1. / 255 ,
    horizontal_flip = True
)

train_data = train_generator.flow_from_directory(
    '/content/train' ,
    target_size = (img_dim,img_dim) ,
    batch_size = batch_size ,
)

val_data = val_generator.flow_from_directory(
    '/content/validation' ,
    target_size = (img_dim,img_dim) ,
    batch_size = batch_size ,
)

test_data = test_generator.flow_from_directory(
    '/content/test' ,
    target_size = (img_dim,img_dim) ,
    batch_size = batch_size ,
)

Found 5179 images belonging to 3 classes.
Found 1033 images belonging to 3 classes.
Found 690 images belonging to 3 classes.


In [33]:
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_dim, img_dim)
else:
    input_shape = (img_dim, img_dim, 3)

model=tf.keras.models.Sequential([
                                  
tf.keras.layers.Conv2D(64,(3,3),activation='relu',input_shape=input_shape),
tf.keras.layers.MaxPool2D(pool_size=(2,2)),

tf.keras.layers.Flatten(),
tf.keras.layers.Dense(128,activation='relu'),
tf.keras.layers.Dense(3,activation='softmax')                                 
                                  
])

model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_11 (Conv2D)           (None, 254, 254, 64)      1792      
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 127, 127, 64)      0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 1032256)           0         
_________________________________________________________________
dense_9 (Dense)              (None, 128)               132128896 
_________________________________________________________________
dense_10 (Dense)             (None, 3)                 387       
Total params: 132,131,075
Trainable params: 132,131,075
Non-trainable params: 0
_________________________________________________________________


In [34]:
model.fit(
    train_data ,
    steps_per_epoch = 5178/batch_size ,
    epochs = 15 ,
    validation_data=val_data ,
    validation_steps = 1032/batch_size ,
    verbose = 1 
)

Epoch 1/15
81/80 [==============================] - 84s 1s/step - loss: 10.5566 - accuracy: 0.7042 - val_loss: 0.4709 - val_accuracy: 0.8412
Epoch 2/15
81/80 [==============================] - 83s 1s/step - loss: 0.3948 - accuracy: 0.8652 - val_loss: 0.3991 - val_accuracy: 0.8732
Epoch 3/15
81/80 [==============================] - 83s 1s/step - loss: 0.2650 - accuracy: 0.9123 - val_loss: 0.3717 - val_accuracy: 0.8829
Epoch 4/15
81/80 [==============================] - 84s 1s/step - loss: 0.2185 - accuracy: 0.9324 - val_loss: 0.3426 - val_accuracy: 0.8916
Epoch 5/15
81/80 [==============================] - 83s 1s/step - loss: 0.1691 - accuracy: 0.9500 - val_loss: 0.3395 - val_accuracy: 0.9071
Epoch 6/15
81/80 [==============================] - 82s 1s/step - loss: 0.1320 - accuracy: 0.9647 - val_loss: 0.3276 - val_accuracy: 0.8945
Epoch 7/15
81/80 [==============================] - 82s 1s/step - loss: 0.1065 - accuracy: 0.9689 - val_loss: 0.3376 - val_accuracy: 0.8984
Epoch 8/15
81/80 [=

In [35]:
test_results = model.evaluate(test_data,batch_size=64)
print(test_results)

11/11 [==============================] - 7s 669ms/step - loss: 0.3268 - accuracy: 0.9087
[0.3267783224582672, 0.908695638179779]


In [27]:
img = cv2.imread('/content/test/covid/007.jpg' , cv2.IMREAD_COLOR)
img = cv2.resize(img, (256,256))
img = np.array(img)
img = tf.expand_dims(img,0)

print(img.shape)
print(input_shape)
    
model(img)


(1, 256, 256, 3)
(256, 256, 3)


<tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[0.9724037 , 0.00989146, 0.01770486]], dtype=float32)>